# EPRB Quantum Circuit Implementation - Fig. 6 Analysis

This notebook demonstrates the implementation of the EPRB (Einstein-Podolsky-Rosen-Bell) quantum circuit from Figure 6 using our NV-Center quantum simulator.

## Circuit Overview

The EPRB circuit implements a two-qubit quantum protocol:
- **Qubit 1**: NV Electron Spin (directly controlled by MW pulses)
- **Qubit 2**: ¹⁴N Nuclear Spin (indirectly controlled via hyperfine coupling)

### Quantum Circuit Diagram

```
Electron |0⟩ ——[X]——[H]——●——[H]——[Rz(α)]——[H]——[M₁]
                         │
Nuclear  |0⟩ ——————[H]——⊕——[H]——[Rz(β)]——[H]——[M₂]
```

Where:
- **X**: π-rotation around x-axis
- **H**: Hadamard gate (creates superposition)
- **●/⊕**: CNOT gate (entangling operation)
- **Rz(α/β)**: Z-rotation with phase α/β
- **M₁/M₂**: Measurement operations


## Physical Implementation

### MW Pulse Sequence Translation

Each quantum gate is implemented using microwave pulses:

| Gate | Implementation | Duration | Phase |
|------|---------------|----------|-------|
| **X₁** (Electron) | MW π-pulse | 35 ns | 0° |
| **H₁** (Electron) | MW π/2-pulse | 17.5 ns | 90° |
| **H₂** (Nuclear) | XY8 DD sequence | 4200 ns | Variable |
| **CNOT** | XY8 DD sequence | 8700 ns | Variable |
| **Rz(α)** | Virtual phase shift | Instant | α |
| **Rz(β)** | Virtual phase shift | Instant | β |

### Key Physics:
- **Direct Control**: MW pulses directly manipulate electron spin
- **Indirect Control**: Nuclear spin controlled via hyperfine coupling
- **Dynamical Decoupling**: XY8 sequences for nuclear spin gates
- **Virtual Gates**: Phase shifts implemented in software


In [ ]:
# Import required libraries
import sys
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display

# Add test directory to path
sys.path.append('../test')

# Import our NV simulator
from main import NVSimulator, load_config, load_system_config

print("📦 Libraries loaded successfully!")
print("🧪 NV Quantum Simulator imported")

## Experiment Configuration

Our experiment is defined in JSON format with precise timing and control parameters:

In [ ]:
# Load the experiment configuration
experiment_config = load_config('../test/experiments/eprb_circuit_fig6.json')
system_config = load_system_config('../test/system.json')

# Display the pulse sequence
print("🎯 EPRB Circuit Pulse Sequence:")
print("=" * 50)

sequence = experiment_config['experiment']['sequence']
for i, step in enumerate(sequence):
    if 'comment' in step:
        print(f"\n{i+1:2d}. {step['comment']}")
    print(f"    Type: {step['type']}")
    print(f"    Start: {step['start_ns']} ns")
    if 'duration_ns' in step:
        print(f"    Duration: {step['duration_ns']} ns")
    if 'omega_rabi_Hz' in step:
        print(f"    Rabi Freq: {step['omega_rabi_Hz']/1e6:.1f} MHz")
    if 'phase_rad' in step:
        print(f"    Phase: {step['phase_rad']:.3f} rad ({step['phase_rad']*180/np.pi:.1f}°)")
    if 'delta_Hz' in step:
        print(f"    Detuning: {step['delta_Hz']/1e6:.1f} MHz")
    if 'shots' in step:
        print(f"    Shots: {step['shots']}")
        
print(f"\n📊 Total experiment time: {experiment_config['experiment']['total_time_ns']/1000:.1f} μs")

## Running the Simulation

Now let's execute the EPRB circuit simulation:

In [ ]:
# Merge configurations and create simulator
config = {**system_config, **experiment_config}
simulator = NVSimulator(config)

print("🚀 Running EPRB Circuit Simulation...")
print("⚛️  Quantum system dimension:", simulator.DIM)
print("🧲 Magnetic field:", simulator.B_vec_G, "G")
print("💎 C13 nuclei:", simulator.N_C)

# Run the experiment
experiment = config['experiment']
results = simulator.run_experiment(experiment)

print("\n✅ Simulation completed!")
print(f"🎯 Final fidelity: {results['fidelity']:.4f}")
print(f"📊 Population ms=0: {results['population_ms0'][-1]:.3f}")
print(f"📊 Population ms=±1: {results['population_ms1'][-1]:.3f}, {results['population_ms_minus1'][-1]:.3f}")

# Generate plots
output_dir = '../results'
simulator.plot_results(results, experiment, save_dir=output_dir)
print(f"\n📈 Plots saved to {output_dir}/")

## Analysis of Results

### Population Dynamics

The simulation tracks the evolution of all three electron spin states throughout the experiment:

In [ ]:
# Plot population dynamics
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

times_us = np.array(results['times_ns']) / 1000  # Convert to microseconds

ax.plot(times_us, results['population_ms_minus1'], 'r-', label='P(ms = -1)', linewidth=2)
ax.plot(times_us, results['population_ms0'], 'b-', label='P(ms = 0)', linewidth=2)
ax.plot(times_us, results['population_ms1'], 'g-', label='P(ms = +1)', linewidth=2)

# Mark key timing points
key_times = [0, 0.05, 0.1, 4.4, 13.2, 18.0, 23.0, 26.0]  # μs
key_labels = ['Start', 'X-gate', 'H-gate', 'CNOT', 'H-gate', 'Final H', 'Meas 1', 'Meas 2']

for t, label in zip(key_times, key_labels):
    ax.axvline(t, color='gray', linestyle='--', alpha=0.5)
    ax.text(t, 0.95, label, rotation=90, ha='right', va='top', fontsize=8)

ax.set_xlabel('Time (μs)')
ax.set_ylabel('Population')
ax.set_title('NV Center Population Dynamics - EPRB Circuit')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.05)

plt.tight_layout()
plt.show()

print("📊 Population dynamics show the effect of each quantum gate")
print("🎯 Key observations:")
print(f"   • Initial state: P(ms=0) = {results['population_ms0'][0]:.3f}")
print(f"   • After X-gate: P(ms=±1) increases")
print(f"   • After H-gates: Superposition states created")
print(f"   • Final state: Mixed populations")

### Photon Count Measurements

The experiment performs two separate measurements:
1. **Electron measurement** (23-25 μs): Direct readout of electron spin
2. **Nuclear measurement** (26-28 μs): Indirect readout via hyperfine mapping

Let's analyze the photon count data:

In [ ]:
# Display photon count results
if isinstance(results['photon_counts'], list):
    print("📊 Multiple photon count measurements detected:")
    
    for i, photon_data in enumerate(results['photon_counts']):
        print(f"\n🔬 Measurement {i+1}:")
        print(f"   • Time window: {photon_data['times_ns'][0]:.0f} - {photon_data['times_ns'][-1]:.0f} ns")
        print(f"   • Bin width: {photon_data['bin_width_ns']} ns")
        print(f"   • Total shots: {photon_data['shots']}")
        print(f"   • Mean counts: {np.mean(photon_data['counts']):.1f} ± {np.std(photon_data['counts']):.1f}")
        print(f"   • Count range: {np.min(photon_data['counts']):.0f} - {np.max(photon_data['counts']):.0f}")
        
        # Quick histogram
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.step(photon_data['times_ns'], photon_data['counts'], where='mid', linewidth=0.7)
        plt.xlabel('Time (ns)')
        plt.ylabel('Photon Counts')
        plt.title(f'Measurement {i+1} - Time Trace')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.hist(photon_data['counts'], bins=20, alpha=0.7, edgecolor='black')
        plt.xlabel('Counts per Bin')
        plt.ylabel('Frequency')
        plt.title(f'Measurement {i+1} - Count Distribution')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
else:
    print("📊 Single photon count measurement:")
    photon_data = results['photon_counts']
    print(f"   • Mean counts: {np.mean(photon_data['counts']):.1f} ± {np.std(photon_data['counts']):.1f}")
    print(f"   • Bin width: {photon_data['bin_width_ns']} ns")
    print(f"   • Total shots: {photon_data['shots']}")

## Display Generated Plots

Let's show the automatically generated plots from our simulation:

In [ ]:
# Display the generated plots
plot_files = [
    '../results/eprb_circuit_electron_photons.png',
    '../results/eprb_circuit_nuclear_photons.png'
]

for i, plot_file in enumerate(plot_files):
    if os.path.exists(plot_file):
        print(f"\n📈 {['Electron', 'Nuclear'][i]} Measurement Results:")
        display(Image(plot_file))
    else:
        print(f"⚠️ Plot file not found: {plot_file}")

## Physics Interpretation

### Quantum Gate Implementation

The EPRB circuit demonstrates several key quantum computing concepts:

1. **Single-Qubit Gates**: Direct MW control of electron spin
   - X-gate: Population transfer between spin states
   - H-gate: Superposition creation
   - Rz-gate: Phase manipulation

2. **Two-Qubit Gates**: Hyperfine-mediated electron-nuclear coupling
   - CNOT: Entangling operation via conditional dynamics
   - Implementation through dynamical decoupling sequences

3. **Quantum Measurements**: Optical readout protocols
   - Electron: Direct fluorescence detection
   - Nuclear: Indirect via hyperfine-selective mapping

### Experimental Challenges

- **Decoherence**: T₁ and T₂* limit gate fidelities
- **Control Precision**: MW amplitude and phase stability
- **Readout Efficiency**: Photon collection and detection
- **Nuclear Control**: Requires long dynamical decoupling sequences

### Performance Metrics

- **Fidelity**: Overlap with target quantum state
- **Contrast**: Signal-to-noise in measurements
- **Timing**: Precision of pulse sequences
- **Reproducibility**: Statistical reliability


## Summary

This notebook demonstrated the implementation of a complete EPRB quantum circuit using NV centers:

✅ **Quantum Circuit**: 10-step protocol with electron-nuclear entanglement  
✅ **MW Control**: Precise pulse sequences for quantum gate implementation  
✅ **Dual Measurements**: Separate electron and nuclear spin readouts  
✅ **Realistic Physics**: Hyperfine coupling, decoherence, and noise effects  
✅ **Data Analysis**: Population dynamics and photon count statistics  

The simulation shows how abstract quantum circuits translate into concrete experimental protocols, bridging the gap between theoretical quantum computing and practical NV center implementations.

### Next Steps

- Optimize gate fidelities through pulse shaping
- Implement error correction protocols
- Scale to larger nuclear spin registers
- Develop quantum algorithms for NV platforms
